In [1]:
from IPython.display import HTML, display
from copy import deepcopy
import random
import os
import openai
import time
import sys
import ast
import json

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from getpass import getpass
import os
import pandas as pd

import pdfkit
import html
from copy import deepcopy
import re

In [2]:
# setting up an OpenAI template on the run
OPENAI_API_KEY = getpass()

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

········


### Setup Model

In [3]:
model = ChatOpenAI(model="gpt-4-0613")
model.temperature = 0.8

### Multi-shot example

In [4]:
example_1 = [ { "from": "human",  "value": "You are web shopping.\nI will give you instructions about what to do.\nYou have to follow the instructions.\nEvery round I will give you an observation and a list of available actions, you have to respond an action based on the state and instruction.\nYou can use search action if search is available.\nYou can click one of the buttons in clickables.\nAn action should be of the following structure:\nsearch[keywords]\nclick[value]\nIf the action is not valid, perform nothing.\nKeywords in search are up to you, but the value in click must be a value in the list of available actions.\nRemember that your keywords in search should be carefully designed.\nYour response should use the following format:\n\nThought:\nI think ... \n\nAction: \nclick[something]" }, { "from": "gpt",  "value": "Ok." }, { "from": "human",  "value": "WebShop [SEP] Instruction: [SEP] i am looking for a 3.25 ounce (pack of 3) of protein serving jerky, and price lower than 50.00 dollars [SEP] Search" }, { "from": "gpt",  "value": "Thought:\nI think I should use the search bar to look for the product I need.\n\nAction:\nsearch[3.25 ounce (pack of 3) protein serving jerky price < 50.00]" }, { "from": "human",  "value": "Instruction: [SEP] i am looking for a 3.25 ounce (pack of 3) of protein serving jerky, and price lower than 50.00 dollars [SEP] Back to Search [SEP] Page 1 (Total results: 50) [SEP] Next > [SEP] B002OFZLOE [SEP] Jack Link's Beef Jerky, Peppered, 3.25-Ounce Bags (Pack of 4) [SEP] $100.0 [SEP] B00P7NYM7M [SEP] Oberto Specialty Meats Teriyaki Beef Jerky, 3.25 Ounce (Pack of 8) [SEP] $59.99 [SEP] B07L3LXDP4 [SEP] Oberto All Natural Teriyaki Beef Jerky, 3.25-Ounce Bag (Pack of 8) [SEP] $100.0 [SEP] B0977H69D1 [SEP] Bridgford Sweet Baby Ray's Original Beef Jerky 3.25oz, 3.25 OZ [SEP] $24.99 [SEP] B005FQAQJO [SEP] Oberto All-Natural Original Beef Jerky, 3.25 Ounce (Pack of 4) [SEP] $36.98 [SEP] B005FQAJWS [SEP] Oberto All-Natural Teriyaki Turkey Jerky, 3.25 Ounce (Pack of 4) [SEP] $33.0 [SEP] B07R3KJ2CY [SEP] Trail’s Best Original Hickory Smoked Beef Jerky 3 Ounce Pack (3 count) [SEP] $13.98 [SEP] B072YXYML2 [SEP] Jack Link Beef Jerky Sweet & Hot - 3.25 Ounce Bag, 1 Count (JERKYS) [SEP] $15.28 [SEP] B000LKU3A6 [SEP] Primal Spirit Vegan Jerky – “Classic Flavor” – Teriyaki, 10 g. Plant Based Protein, Certified Non-GMO, No Preservatives, Sports Friendly Packaging (24 Pack, 1 oz) [SEP] $33.99 [SEP] B000LKU3K6 [SEP] Primal Spirit Vegan Jerky – “Classic Flavor” – Hickory Smoked, 10 g. Plant Based Protein, Certified Non-GMO, No Preservatives, Sports Friendly Packaging (24 Pack, 1 oz) [SEP] $33.99" }, { "from": "gpt",  "value": "Thought:\nI think I should click on the 'B0977H69D1' option, as it meets my criteria of a 3.25 ounce (pack of 3) protein serving jerky with a price lower than $50.00.\n\nAction:\nclick[B0977H69D1]" }, { "from": "human",  "value": "Instruction: [SEP] i am looking for a 3.25 ounce (pack of 3) of protein serving jerky, and price lower than 50.00 dollars [SEP] Back to Search [SEP] < Prev [SEP] flavor name [SEP] honey bbq [SEP] original [SEP] peppered [SEP] sweet n spicy [SEP] sweet teriyaki [SEP] variety pack [SEP] size [SEP] 3.25 ounce (pack of 2) [SEP] 3.25 ounce (pack of 3) [SEP] 3.25 ounce (pack of 4) [SEP] 3.25 ounce (pack of 5) [SEP] Bridgford Sweet Baby Ray's Original Beef Jerky 3.25oz, 3.25 OZ [SEP] Price: $24.99 [SEP] Rating: N.A. [SEP] Description [SEP] Features [SEP] Reviews [SEP] Buy Now" }, { "from": "gpt",  "value": "Thought:\nI think I should click on the '3.25 ounce (pack of 3)' option to select the size I need for the jerky.\n\nAction:\nclick[3.25 ounce (pack of 3)]" }, { "from": "human",  "value": "Instruction: [SEP] i am looking for a 3.25 ounce (pack of 3) of protein serving jerky, and price lower than 50.00 dollars [SEP] Back to Search [SEP] < Prev [SEP] flavor name [SEP] honey bbq [SEP] original [SEP] peppered [SEP] sweet n spicy [SEP] sweet teriyaki [SEP] variety pack [SEP] size [SEP] 3.25 ounce (pack of 2) [SEP] 3.25 ounce (pack of 3) [SEP] 3.25 ounce (pack of 4) [SEP] 3.25 ounce (pack of 5) [SEP] Bridgford Sweet Baby Ray's Original Beef Jerky 3.25oz, 3.25 OZ [SEP] Price: $24.99 [SEP] Rating: N.A. [SEP] Description [SEP] Features [SEP] Reviews [SEP] Buy Now" }, { "from": "gpt",  "value": "Thought:\nI think I should click on the 'Buy Now' button to purchase the Bridgford Sweet Baby Ray's Original Beef Jerky.\n\nAction:\nclick[Buy Now]" } ]

In [5]:
one_shot_messages = []
for dialog in example_1:
    if dialog['from'] == 'human':
        msg = HumanMessage(content=dialog['value'])
    elif dialog['from'] == 'gpt':
        msg = AIMessage(content=dialog['value'])
    one_shot_messages.append(msg)
    

In [30]:
example_1[2]

{'from': 'human',
 'value': 'WebShop [SEP] Instruction: [SEP] i am looking for a 3.25 ounce (pack of 3) of protein serving jerky, and price lower than 50.00 dollars [SEP] Search'}

### WebShop Environment

In [32]:
environment_prompt_template = '''
You will act as the interface for an online shopping platform. An agent is participating in a simulated web shopping experience, and your role is to present the environment and react to the agent's actions. When the agent issues an action, such as a search query or a selection from available clickables, you must generate a response that realistically reflects what they would encounter on an e-commerce website.

For search actions, the agent will provide specific keywords and possibly additional parameters like price limits. Your response should be a list of relevant products with identifiers, descriptions, and prices, formatted in a straightforward, readable manner without using any special markup or separators other than simple punctuation. Include at least five product options in your response to offer a variety.

For click actions, the agent will choose an option from the list of clickables you've provided. These clickables could be product identifiers, page navigation options like "Next" or "Back to Search," or other interactive elements you've introduced in the simulation. Respond with the outcome of their action, which could be the opening of a product page, navigating to another list of products, or other plausible results of clicking a button on a web page.

Remember to maintain a coherent state within the simulation. If the agent performs a search, the following clickable actions should correspond to the products from the search results. If the agent has accomplished their task, YOU MUST call the success function: "success('[why task was successful]')".
'''

first_page = '''
Please start by providing the agent their task and the search page (after this will be only agent messages): 
{}
'''

ws_agent_prompt = 'You are web shopping.\nI will give you instructions about what to do.\nYou have to follow the instructions.\nEvery round I will give you an observation and a list of available actions, you have to respond an action based on the state and instruction.\nYou can use search action if search is available.\nYou can click one of the buttons in clickables.\nAn action should be of the following structure:\nsearch[keywords]\nclick[value]\nIf the action is not valid, perform nothing.\nKeywords in search are up to you, but the value in click must be a value in the list of available actions.\nRemember that your keywords in search should be carefully designed.\nYour response should use the following format:\n\nThought:\nI think ... \n\nAction: \nclick[something]'

In [40]:
example_task = one_shot_messages[2].content

In [21]:
one_shot_environment_messages = [
    HumanMessage(content=environment_prompt_template),
    AIMessage(content="OK. I'll follow your instructions and try my best to act as the interface for an online shopping platform."),
    HumanMessage(content=first_page.format(example_task))
]


In [22]:
one_shot_environment_messages

[HumanMessage(content='\nYou will act as the interface for an online shopping platform. An agent is participating in a simulated web shopping experience, and your role is to present the environment and react to the agent\'s actions. When the agent issues an action, such as a search query or a selection from available clickables, you must generate a response that realistically reflects what they would encounter on an e-commerce website.\n\nFor search actions, the agent will provide specific keywords and possibly additional parameters like price limits. Your response should be a list of relevant products with identifiers, descriptions, and prices, formatted in a straightforward, readable manner without using any special markup or separators other than simple punctuation. Include at least five product options in your response to offer a variety.\n\nFor click actions, the agent will choose an option from the list of clickables you\'ve provided. These clickables could be product identifiers

In [26]:
for dialog in example_1[2:]:
    if dialog['from'] == 'human':
        msg = AIMessage(content=dialog['value'])
    elif dialog['from'] == 'gpt':
        msg = HumanMessage(content=dialog['value'])
    one_shot_environment_messages.append(msg)
    
    
one_shot_environment_messages.append(AIMessage(content="success('agent purchased Bridgford Sweet Baby Ray's Original Beef Jerky.')"))  

In [50]:
agent_messages = [
    HumanMessage(content=ws_agent_prompt),
    AIMessage(content='Ok.')
]




In [51]:
agent_messages

[HumanMessage(content='You are web shopping.\nI will give you instructions about what to do.\nYou have to follow the instructions.\nEvery round I will give you an observation and a list of available actions, you have to respond an action based on the state and instruction.\nYou can use search action if search is available.\nYou can click one of the buttons in clickables.\nAn action should be of the following structure:\nsearch[keywords]\nclick[value]\nIf the action is not valid, perform nothing.\nKeywords in search are up to you, but the value in click must be a value in the list of available actions.\nRemember that your keywords in search should be carefully designed.\nYour response should use the following format:\n\nThought:\nI think ... \n\nAction: \nclick[something]', additional_kwargs={}, example=False),
 AIMessage(content='Ok.', additional_kwargs={}, example=False)]

In [52]:
task = "Please start by providing the agent their task and the search page (after this will be only agent messages): \ncontent='WebShop [SEP] Instruction: [SEP] i am looking for a 3.25 ounce (pack of 3) of protein serving jerky, and price lower than 50.00 dollars [SEP] Search"

environment_messages = one_shot_environment_messages + [
    HumanMessage(content=environment_prompt_template),
    AIMessage(content="OK. I'll follow your instructions and try my best to act as the interface for an online shopping platform."),
    HumanMessage(content=first_page.format(task))
]

In [53]:
environment_result = model.predict_messages(environment_messages)

In [54]:
environment_result.content

'WebShop [SEP] Instruction: [SEP] i am looking for a 3.25 ounce (pack of 3) of protein serving jerky, and price lower than 50.00 dollars [SEP] Search'

In [55]:
environment_messages.append(environment_result)
agent_messages.append(HumanMessage(content=environment_result.content))

In [56]:
num_turns = 10

for i in range(num_turns):
    agent_response = model.predict_messages(agent_messages)
    print(agent_response.content)
    agent_result = agent_response.content
    environment_messages.append(HumanMessage(content=agent_result))
    agent_messages.append(agent_response)
    environment_result = model.predict_messages(environment_messages)
    environment_messages.append(environment_result)
    print(environment_result.content)
    if "success(" in environment_result.content:
        break
    agent_messages.append(HumanMessage(content=environment_result.content))

Thought:
I need to search for a 3.25 ounce, pack of 3 protein serving jerky that costs less than $50. 

Action: 
search[3.25 ounce pack of 3 protein serving jerky under $50]
Here are some options for your search:

1. Product ID: JKY123 - Jack Link's Original Protein Snack Jerky, 3.25 Ounce (Pack of 3) - $45.99
2. Product ID: OBT890 - Oberto All-Natural Original Beef Jerky, 3.25 Ounce (Pack of 3) - $39.99
3. Product ID: TBY654 - Tillamook Country Smoker All Natural, Real Hardwood Smoked Beef Jerky, 3.25 Ounce (Pack of 3) - $42.99
4. Product ID: PRT567 - Primal Strips Meatless Vegan Jerky-Variety Gift Pack Sampler, 3.25 Ounce (Pack of 3) - $35.99
5. Product ID: KRA345 - Krave Gourmet Jerky, 3.25 Ounce (Pack of 3) - $49.99

You can click on a product ID for more information.
Thought:
Based on the options provided, all of them meet the criteria. However, the cheapest one is the Primal Strips Meatless Vegan Jerky. 

Action: 
click[PRT567]
You've selected the Primal Strips Meatless Vegan Jer

In [49]:
agent_messages

[HumanMessage(content='You are web shopping.\nI will give you instructions about what to do.\nYou have to follow the instructions.\nEvery round I will give you an observation and a list of available actions, you have to respond an action based on the state and instruction.\nYou can use search action if search is available.\nYou can click one of the buttons in clickables.\nAn action should be of the following structure:\nsearch[keywords]\nclick[value]\nIf the action is not valid, perform nothing.\nKeywords in search are up to you, but the value in click must be a value in the list of available actions.\nRemember that your keywords in search should be carefully designed.\nYour response should use the following format:\n\nThought:\nI think ... \n\nAction: \nclick[something]', additional_kwargs={}, example=False),
 AIMessage(content='Ok.', additional_kwargs={}, example=False),
 HumanMessage(content='WebShop [SEP] Instruction: [SEP] i am looking for a 3.25 ounce (pack of 3) of protein servi

In [57]:
environment_messages

[HumanMessage(content='\nYou will act as the interface for an online shopping platform. An agent is participating in a simulated web shopping experience, and your role is to present the environment and react to the agent\'s actions. When the agent issues an action, such as a search query or a selection from available clickables, you must generate a response that realistically reflects what they would encounter on an e-commerce website.\n\nFor search actions, the agent will provide specific keywords and possibly additional parameters like price limits. Your response should be a list of relevant products with identifiers, descriptions, and prices, formatted in a straightforward, readable manner without using any special markup or separators other than simple punctuation. Include at least five product options in your response to offer a variety.\n\nFor click actions, the agent will choose an option from the list of clickables you\'ve provided. These clickables could be product identifiers

### Get Tasks from AgentInstruct Data

In [58]:
import pandas as pd
df = pd.read_parquet('~/spikes/notebook_spikes/agentinstruct/webshop-00000-of-00001-9f2ae60445e11b4e.parquet')
print(df)

                                         conversations           id
0    [{'from': 'human', 'loss': None, 'value': 'You...    webshop_0
1    [{'from': 'human', 'loss': None, 'value': 'You...    webshop_1
2    [{'from': 'human', 'loss': None, 'value': 'You...    webshop_2
3    [{'from': 'human', 'loss': None, 'value': 'You...    webshop_3
4    [{'from': 'human', 'loss': None, 'value': 'You...    webshop_4
..                                                 ...          ...
346  [{'from': 'human', 'loss': None, 'value': 'You...  webshop_346
347  [{'from': 'human', 'loss': None, 'value': 'You...  webshop_347
348  [{'from': 'human', 'loss': None, 'value': 'You...  webshop_348
349  [{'from': 'human', 'loss': None, 'value': 'You...  webshop_349
350  [{'from': 'human', 'loss': None, 'value': 'You...  webshop_350

[351 rows x 2 columns]


In [59]:
print(df.iloc[2]["conversations"][2]['value'])

WebShop [SEP] Instruction: [SEP] coney island classics butter me up popcorn, brings back memories of my childhood. in addition to having dietary fiber and gluten free. please select for me, and price lower than 30.00 dollars [SEP] Search


In [60]:
def process(data):
    return data['conversations'][2]['value']

In [61]:
task_and_environment = df.apply(process, axis=1).tolist()

In [62]:
len(task_and_environment)

351

In [63]:
task_and_environment[200]

'WebShop [SEP] Instruction: [SEP] i am looking for a game joystick that has a usb port and must be the color black, and price lower than 50.00 dollars [SEP] Search'

### Create Training Data Conversations

In [98]:
num_turns = 20

#all_agents = []
#all_environments = []
start = 351

for x in range(start,start+len(task_and_environment)): 
    task = task_and_environment[x]
    print("Task {}:".format(x), task)
    agent_messages = [
        HumanMessage(content=ws_agent_prompt),
        AIMessage(content='Ok.')
    ]
    environment_messages = one_shot_environment_messages + [
        HumanMessage(content=environment_prompt_template),
        AIMessage(content="OK. I'll follow your instructions and try my best to act as the interface for an online shopping platform."),
        HumanMessage(content=first_page.format(task)),
        AIMessage(content=task)
    ]
    agent_messages.append(HumanMessage(content=task))
    for i in range(num_turns):
        agent_response = model.predict_messages(agent_messages)
        print(agent_response.content)
        agent_messages.append(agent_response)
        environment_messages.append(HumanMessage(content=agent_response.content))
        environment_result = model.predict_messages(environment_messages)
        environment_messages.append(environment_result)
        print(environment_result.content)
        if "success(" in environment_result.content:
            break
        agent_messages.append(HumanMessage(content=environment_result.content))
    all_agents.append(deepcopy(agent_messages))
    all_environments.append(deepcopy(environment_messages))

IndexError: list index out of range

In [97]:
len(all_agents)

351

In [99]:
conversations_ = []
for chat in all_agents:
    conversation_ = []
    for side in chat:
        if side.type == "human":
            side_type = "human"
        else:
            side_type = "gpt"
        conversation_.append({"from": side_type, "value":side.content})
    conversations_.append(conversation_ )
        

In [100]:
with open('./synthetic/webshop-raw.jsonl', 'w') as file:
    for conversation in conversations_:
        json_str = json.dumps(conversation)
        file.write(json_str + "\n")